# Dados UFRN - Análise


## Descrição do projeto

## Descrição dos dados


## Autores


## Vídeo explicativo



Importando bibliotecas a serem usadas

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#conda install -c conda-forge seaborn

Abertura de dados

In [33]:
#Carregando arquivo referente as unidades
data_unidades = pd.read_csv("Dados/unidades/unidades.csv", sep=';',
                            usecols=["id_unidade", "nome_unidade", "municipio","id_unidade_gestora"])

data_unidades = data_unidades[data_unidades.municipio == "NATAL"]


#Selecionando somente os institutos e os centros
centro_ou_inst = [nome.startswith("CENTRO") or nome.startswith("INSTITUTO") for nome in data_unidades.nome_unidade]
data_unidades_ci = data_unidades[centro_ou_inst]

for i in range(len(centro_ou_inst)):
    if centro_ou_inst[i]:
        data_unidades.iloc[i, data_unidades.columns.get_loc("id_unidade_gestora")] = data_unidades.iloc[i, data_unidades.columns.get_loc("id_unidade")]

# data_unidades = data_unidades_ci.drop(columns=["id_unidade_gestora"], axis=1)
# data_unidades["id_unidade_gestora"] = data_unidades_ci.id_unidade

#Carregando arquivo referente aos gastos
data_gastos = pd.read_csv("Dados/unidades/gastos-por-unidade.csv", 
                          sep=';',
                          usecols=["id_unidade", "natureza_despesa", "valor"])

#Convertendo coluna de String para float
data_gastos.valor = data_gastos.valor.replace('[R | ' ' | \$,.]', '', regex=True).astype(float) / 100

#Unindo as tabelas
dataM_unit_gas = pd.merge(data_unidades_ci, data_gastos, on="id_unidade")

#Criando pivot table para somar gasto por unidade
ptable = pd.pivot_table(dataM_unit_gas, index=["id_unidade", "nome_unidade"], values=["valor"], aggfunc=np.sum)

#Salvando essa pivot table num DataFrame
data_gastos_ok = pd.DataFrame(ptable.to_records()).rename(columns={'id_unidade': 'id_unidade_gestora'})

link_unidades = data_unidades[["id_unidade", "id_unidade_gestora"]]

print(data_gastos_ok)

   id_unidade_gestora                                nome_unidade  \
0                 439        CENTRO DE CIENCIAS EXATAS E DA TERRA   
1                 440                       CENTRO DE BIOCIENCIAS   
2                 441                 CENTRO DE CIENCIAS DA SAUDE   
3                 442  CENTRO DE CIENCIAS HUMANAS, LETRAS E ARTES   
4                 443        CENTRO DE CIENCIAS SOCIAIS APLICADAS   
5                 445                        CENTRO DE TECNOLOGIA   
6                5965                          CENTRO DE EDUCACAO   
7                6026                        INSTITUTO DO CEREBRO   
8                6069                 INSTITUTO METROPOLE DIGITAL   
9                6919     INSTITUTO DE MEDICINA TROPICAL - IMT-RN   

          valor  
0  1.043835e+08  
1  6.436635e+07  
2  1.184980e+08  
3  6.963006e+07  
4  8.373748e+07  
5  7.818721e+07  
6  1.867217e+07  
7  3.303266e+07  
8  4.944037e+07  
9  2.767012e+06  


In [5]:
#Carregando arquivo dos docentes (link entre turmas e unidades gestoras)
link_docentes = pd.read_csv("Dados/docentes/docentes.csv",
                           sep=';',
                           usecols=["id_servidor", "id_unidade_lotacao"])

In [ ]:
#Carregando arquivos de semestre e linkando

for i in range(2016,2018):
    for j in range(1,3):
        #Carregando strings com o caminho dos dados a serem carregados
        file_matricula = "Dados/matriculas/" + (str(i) + str(j)) + ".csv"
        file_turmas = "Dados/turmas/" + (str(i) + str(j)) + ".csv"
        
        #Carregando arquivo de matriculas de certo semestre para DataFrame
        data_matriculas = pd.read_csv(file_matricula, sep=';',
                                      usecols=["id_turma", "discente", "descricao"])
        #id_turma: necessario para linkar com "turmas"
        #discente: necessario para contagem
        #descricao: o que vamos contar
        
        #Separando valores de descricao irrelevantes para o problema
        tags_indesejadas = ["INDEFERIDO", "AGUARDANDO DEFERIMENTO", "EM ESPERA", 
                            "CUMPRIU", "TRANSFERIDO", "DISPENSADO", "EXCLUIDA"]
        
        #Selecionando apenas os valores nao irrelevantes (relevantes)
        data_matriculas = data_matriculas[~data_matriculas.descricao.isin(tags_indesejadas)]
        
        #Criando mapa para classificar APROVADO ou APROVADO POR NOTA como feedback positivo
        #E classificar todas outras tags como feedback negativo
        map_dict = {}
        for desc in data_matriculas.descricao:
            if desc == "APROVADO POR NOTA" or desc == "APROVADO":
                map_dict[desc] = 1
            else:
                map_dict[desc] = 0
            
        #Atualizar valores das tags com valores mapeados
        data_matriculas.descricao = data_matriculas.descricao.map(map_dict)
        
        #Carregando arquivo de turmas de certo semestre para DataFrame
        data_turmas = pd.read_csv(file_turmas, sep=';',
                                  usecols=["id_turma", "id_docente_interno", "nivel_ensino",
                                           "ano", "periodo","situacao_turma"])
        #id_turma: necessario para linkar com "matriculas"
        #id_docente_interno: necessario para linkar com "docentes"
        #nivel_ensino: restringir para GRADUACAO
        #ano: restringir para ano analisado
        #periodo: restringir para periodo analisado
        #situacao_turma: restringir para turmas consolidadas
        
        #Restringindo a busca para o ano procurado
        data_turmas = data_turmas[data_turmas.ano == i]
        
        #Restringindo a busca para o periodo procurado
        data_turmas = data_turmas[data_turmas.periodo == j]
        
        #Restringindo a busca para o curso superior
        data_turmas = data_turmas[data_turmas.nivel_ensino == "GRADUAÇÃO"]
        
        #Restringindo a busca para turmas consolidadas
        data_turmas = data_turmas[data_turmas.situacao_turma == "CONSOLIDADA"]
        
        #Dando merge nas tabelas turmas e matriculas
        dataM_mat_tur = pd.merge(data_turmas, data_matriculas, on="id_turma")
        
        #Criando pivot table para contar reprovacoes, aprovacoes...
        ptable = pd.pivot_table(dataM_mat_tur, index=["id_turma", "id_docente_interno", "descricao"],
                                values="ano", aggfunc="count").rename(columns={'ano': 'Contagem'})
        
        #Salvando essa pivot table num DataFrame
        data_cont_situ = pd.DataFrame(ptable.to_records())        
        
        #Corrigindo nome de coluna, para possibilitar merge
        data_cont_situ.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)
        
        #Executando merge
        dataM_mt_doc = pd.merge(data_cont_situ, link_docentes, on="id_servidor")
        
        #Criando pivot table para agregar contagens por unidade e entao por turma
        ptable = pd.pivot_table(dataM_mt_doc, index=["id_unidade_lotacao", "id_turma", "descricao"],
                               values="Contagem")
        
        #Salvando essa pivot table num DataFrame
        data_cont_unit = pd.DataFrame(ptable.to_records())
        
        #Adicionando nova coluna, referente a porcentagem de valores em "descricao" por "id_turma"
        data_cont_unit["% da turma"] = (data_cont_unit.Contagem / 
                                        data_cont_unit.groupby(["id_unidade_lotacao","id_turma"])
                                        .Contagem.transform(sum) * 100)
        
        ptable = pd.pivot_table(data_cont_unit, index=["id_unidade_lotacao", "id_turma" ,"descricao"],
                        values="% da turma", aggfunc=np.mean)
        
        #Salvando essa pivot table num DataFrame
        data_cont_unit = pd.DataFrame(ptable.to_records())
        
        #Adicionando valores 0% nas turmas com 100% de aprovacao (ou reprovacao) para poder calcular a media
        for (unidade, turma), _count in data_cont_unit.groupby(["id_unidade_lotacao", "id_turma"])["descricao"].count().iteritems():
            if _count == 1:
                soh_aprovado_ou_nao = data_cont_unit[(data_cont_unit.id_turma == turma) & 
                                  (data_cont_unit.id_unidade_lotacao == unidade)]["descricao"]
   
                temp_df = pd.DataFrame(data={"id_unidade_lotacao": unidade,
                                             "id_turma": turma,
                                             "descricao": 1 - soh_aprovado_ou_nao,
                                             "% da turma": 0.0}, columns=data_cont_unit.columns)
                
                data_cont_unit = data_cont_unit.append(temp_df, ignore_index=True)
        
        #Calculando a media de aprovacao por unidade
        ptable = pd.pivot_table(data_cont_unit, index=["id_unidade_lotacao" ,"descricao"],
                        values="% da turma", aggfunc=np.mean)
        
        #Salvando essa pivot table num DataFrame
        data_aprov_unit = pd.DataFrame(ptable.to_records()).rename(columns={"id_unidade_lotacao": "id_unidade"})
        
        data_aprov_unit = pd.merge(link_unidades, data_aprov_unit, on="id_unidade")
        
        #Calculando a media de aprovacao por unidade gestora
        ptable = pd.pivot_table(data_aprov_unit, index=["id_unidade_gestora" ,"descricao"],
                        values="% da turma", aggfunc=np.mean)
        
        #Salvando essa pivot table num DataFrame
        data_aprov_unit = pd.DataFrame(ptable.to_records())
        
        data_gasto_aprov = pd.merge(data_gastos_ok, data_aprov_unit, on="id_unidade_gestora")
        
        print(data_gasto_aprov)
    
             
            
            
        

    id_unidade_gestora                                nome_unidade  \
0                  439        CENTRO DE CIENCIAS EXATAS E DA TERRA   
1                  439        CENTRO DE CIENCIAS EXATAS E DA TERRA   
2                  440                       CENTRO DE BIOCIENCIAS   
3                  440                       CENTRO DE BIOCIENCIAS   
4                  441                 CENTRO DE CIENCIAS DA SAUDE   
5                  441                 CENTRO DE CIENCIAS DA SAUDE   
6                  442  CENTRO DE CIENCIAS HUMANAS, LETRAS E ARTES   
7                  442  CENTRO DE CIENCIAS HUMANAS, LETRAS E ARTES   
8                  443        CENTRO DE CIENCIAS SOCIAIS APLICADAS   
9                  443        CENTRO DE CIENCIAS SOCIAIS APLICADAS   
10                 445                        CENTRO DE TECNOLOGIA   
11                 445                        CENTRO DE TECNOLOGIA   
12                5965                          CENTRO DE EDUCACAO   
13                59

In [1]:
sns.set()
data_gastos
# The four preset contexts, in order of relative size, are paper, notebook, talk, and poster
sns.set_context("notebook")
data_gastos.valor = data_gastos.valor.astype(float)
# plot a univariate distribution of observations.
#sns.kdeplot(int(data_gastos["valor"]))

NameError: name 'sns' is not defined